In [1]:
import pandas as pd
import easyocr
import os
from tqdm.notebook import tqdm
import torch
import numpy as np

rootPath = "D:/DTU-D/4/Fagprojekt/Data"
csvName = "labels.csv"
csvName2 = "labels2.csv"

reader = easyocr.Reader(['en'])
data = pd.concat([
    pd.read_csv(os.path.join(rootPath,csvName)),
    pd.read_csv(os.path.join(rootPath,csvName2))
])
batch = 1
# data.drop(data.index[[1711]], inplace=True) #GIF
data.drop(data.index[[3033,3034,3035]], inplace=True) #Corrupted pictures
rows = data.shape[0]

In [2]:
results = []
ground_truth = []
correctIn = 0
for i in tqdm(range(rows)):
    # Primitive attempt at batching
    if i % batch == 0:
        del reader
        torch.cuda.empty_cache()
        reader = easyocr.Reader(['en'])

    image_path = data.iloc[i]['image_path'].lower()
    if image_path.endswith("gif") or image_path.endswith("pdf"):
        continue
    # The replaced letters are NOT the same. There is a mismatch between the csv file and the actual picture name
    if "ä" in image_path:
        image_path = image_path.replace("ä", "ä")
    if "å" in image_path:
        image_path = image_path.replace("å", "å")
    if "ö" in image_path:
        image_path = image_path.replace("ö", "ö")
    path = os.path.join(rootPath,image_path)

    result = reader.readtext(path, paragraph=True)
    results.append(result)

    y = data.iloc[i]['meter_number']
    ground_truth.append(y)
    for j in range(len(result)):
        if y in result[j]:
            correctIn += 1
            break

In [3]:
# import cv2
# result = reader.readtext(path, paragraph=True)
# # cv2.imdecode(np.fromfile(path, np.uint8), cv2.IMREAD_UNCHANGED).shape

In [ ]:
print(f"pretrained accuracy: {(correctIn/len(data.project_id.unique()))*100}")

In [ ]:
# np.savez("Variables_results_08_04_2022_detailsOn_paragraphTrue",
#          correctIn=correctIn,
#          ground_truth=ground_truth
#          results=np.array(results,dtype=object))

In [ ]:
# # Rename files
# tochange = [29,45,133,135,136,180,186,203,303,347,349,369,379,429,514,562,566,644,645,713,821,946,1008,1073,1074,1087,1211,1212,1233,1325,1558,1568,1573,1574,1577,1611,1743,1900,372,908,958,959,992,993,1061,1362,1651,1671,1675,1735]
#
# for i in tochange:
#     image_path = data.iloc[i]['image_path']
#     newName = os.path.join(rootPath,image_path.replace("å","aa").replace("ø","oo").replace("æ","ae"))
#     oldFileName = os.listdir(os.path.join(rootPath,os.path.dirname(image_path)))[0]
#     oldName = os.path.join(rootPath,os.path.dirname(image_path),oldFileName)
#     os.rename(oldName,newName)